In [ ]:
import networkx as nx
import pandas as pd
import sys
from PySide6.QtWidgets import QApplication, QDialog, QVBoxLayout, QWidget
from PySide6.QtWebEngineWidgets import QWebEngineView


class CentralityAnalysis:
    def __init__(self):
        super().__init__()
        # Initialize graph variable
        self.G = None
        
    def generate_graph(self):
        # Read data and create graph
        df = pd.read_csv('1YOK.cif_ringEdges', sep='\t')
        subset_df = df.loc[(df['NodeId1'].str.contains('A:')) & (df['NodeId2'].str.contains('A:'))]
        self.G = nx.from_pandas_edgelist(subset_df, 'NodeId1', 'NodeId2', create_using=nx.Graph())
        
    def generate_eigenVals(self):
        # Check if graph is generated
        if self.G is None:
            raise ValueError("Graph not generated. Call generate_graph() first.")
        
        eigen_centr = nx.eigenvector_centrality_numpy(self.G)
        eigendf = pd.DataFrame(list(eigen_centr.items()), columns=['node', 'value'])
        eigendf['norm_val'] = eigendf['value'] / eigendf['value'].max()
        return eigendf
        
    def generate_closeVals(self):
        if self.G is None:
            raise ValueError("Graph not generated. Call generate_graph() first.")
        
        close_centr = nx.closeness_centrality(self.G)
        closedf = pd.DataFrame(list(close_centr.items()), columns=['node', 'value'])
        closedf['norm_val'] = closedf['value'] / closedf['value'].max()
        return closedf
        
    def generate_degrVals(self):
        if self.G is None:
            raise ValueError("Graph not generated. Call generate_graph() first.")
        
        degr_centr = nx.degree_centrality(self.G)
        degrdf = pd.DataFrame(list(degr_centr.items()), columns=['node', 'value'])
        degrdf['norm_val'] = degrdf['value'] / degrdf['value'].max()
        return degrdf
        
    def generate_betwVals(self):
        if self.G is None:
            raise ValueError("Graph not generated. Call generate_graph() first.")
        
        betw_centr = nx.betweenness_centrality(self.G)
        betwdf = pd.DataFrame(list(betw_centr.items()), columns=['node', 'value'])
        betwdf['norm_val'] = betwdf['value'] / betwdf['value'].max()
        return betwdf
            
    def generate_edgeBetw(self):
        if self.G is None:
            raise ValueError("Graph not generated. Call generate_graph() first.")
        
        edge_betw = nx.edge_betweenness_centrality(self.G)
        edge_betwdf = pd.DataFrame(list(edge_betw.items()), columns=['Pair', 'Edge Value'])
        edge_betwdf[['Node1', 'Node2']] = pd.DataFrame(edge_betwdf['Pair'].tolist(), index=edge_betwdf.index)
        edge_betwdf = edge_betwdf[['Node1', 'Node2', 'Edge Value']]
        edge_betwdf['Norm Value'] = edge_betwdf['Edge Value'] / edge_betwdf['Edge Value'].max()
        return edge_betwdf
    
class PlotlyGraph(QWidget):
    def __init__(self):
        super().__init__()

    def generate_plot(self, html_content):
        dialog = QDialog(self)
        dialog.setWindowTitle("Plot Window")
        dialog.setGeometry(200, 200, 800, 600)

        layout = QVBoxLayout(dialog)

        web_view = QWebEngineView()
        web_view.setHtml(html_content)
        layout.addWidget(web_view)

        dialog.setLayout(layout)
        dialog.exec_()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    plotter = PlotlyGraph()

    # Instantiate CentralityAnalysis
    newGraph = CentralityAnalysis()
    newGraph.generate_graph()
    
    eigenvals = newGraph.generate_eigenVals()
    eigenvals['node'] = eigenvals['node'].str.replace(r'\D', '', regex=True)
    eigenvals['node'] = pd.to_numeric(eigenvals['node'])
    eigenvals = eigenvals.sort_values(by='node', ascending=True)
    eigenvals = eigenvals.drop(eigenvals.index[-1])
    
    betwvals = newGraph.generate_betwVals()
    betwvals['node'] = betwvals['node'].str.replace(r'\D', '', regex=True)
    betwvals['node'] = pd.to_numeric(betwvals['node'])
    betwvals = betwvals.sort_values(by='node', ascending=True)
    betwvals = betwvals.drop(betwvals.index[-1])

    degrvals = newGraph.generate_degrVals()
    degrvals['node'] = degrvals['node'].str.replace(r'\D', '', regex=True)
    degrvals['node'] = pd.to_numeric(degrvals['node'])
    degrvals = degrvals.sort_values(by='node', ascending=True)
    degrvals = degrvals.drop(degrvals.index[-1])

    closevals = newGraph.generate_closeVals()
    closevals['node'] = closevals['node'].str.replace(r'\D', '', regex=True)
    closevals['node'] = pd.to_numeric(closevals['node'])
    closevals = closevals.sort_values(by='node', ascending=True)
    closevals = closevals.drop(closevals.index[-1])

    # Your HTML content goes here
    line_plot_html = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            .plot-div {{
                width: 800px;
                height: 300px;
            }}
        </style>
    </head>
    <body>
        <div id="plot1" class="plot-div"></div>
        <div id="plot2" class="plot-div"></div>
        <div id="plot3" class="plot-div"></div>
        <div id="plot4" class="plot-div"></div>
        <script>
            var trace1 = {{
                x: {list(eigenvals['node'])},
                y: {list(eigenvals['norm_val'])},
                mode: 'lines',
                type: 'scatter',
                name: 'Eigenvals',
                marker: {{ color: '#32CD32' }}
            }};
            var layout1 = {{
                title: 'Eigenvals Plot',
                xaxis: {{ title: 'Node' }},
                yaxis: {{ title: 'Norm Val', rangemode: 'tozero' }}
            }};
            Plotly.newPlot('plot1', [trace1], layout1);

            var trace2 = {{
                x: {list(betwvals['node'])},
                y: {list(betwvals['norm_val'])},
                mode: 'lines',
                type: 'scatter',
                name: 'Betwvals',
                marker: {{ color: '#FF5733' }}
            }};
            var layout2 = {{
                title: 'Betwvals Plot',
                xaxis: {{ title: 'Node' }},
                yaxis: {{ title: 'Norm Val', rangemode: 'tozero' }}
            }};
            Plotly.newPlot('plot2', [trace2], layout2);

            var trace3 = {{
                x: {list(degrvals['node'])},
                y: {list(degrvals['norm_val'])},
                mode: 'lines',
                type: 'scatter',
                name: 'Degrvals',
                marker: {{ color: '#0000FF' }}
            }};
            var layout3 = {{
                title: 'Degrvals Plot',
                xaxis: {{ title: 'Node' }},
                yaxis: {{ title: 'Norm Val', rangemode: 'tozero' }}
            }};
            Plotly.newPlot('plot3', [trace3], layout3);

            var trace4 = {{
                x: {list(closevals['node'])},
                y: {list(closevals['norm_val'])},
                mode: 'lines',
                type: 'scatter',
                name: 'Closevals',
                marker: {{ color: '#FFA500' }}
            }};
            var layout4 = {{
                title: 'Closevals Plot',
                xaxis: {{ title: 'Node' }},
                yaxis: {{ title: 'Norm Val', rangemode: 'tozero' }}
            }};
            Plotly.newPlot('plot4', [trace4], layout4);
        </script>
    </body>
    </html>
    '''

    plotter.generate_plot(line_plot_html)
    sys.exit(app.exec())